The bot code

In [ ]:
!pip install -q langchain langchain-groq
!pip install -q python-telegram-bot
!pip install -q langchain
!pip install -q --upgrade replicate
!pip install -q Image
!pip install -q transformers
!pip install -q telebot
!pip install -q SpeechRecognition
!pip install -qq lightning
!pip install -q telethon
!pip install -q pydub
!pip install -q deepface
!pip install -q --upgrade Pillow
!pip install -q fer
!pip install -q pydub
!pip install -q audiosegment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.7/618.7 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (se

In [ ]:
from transformers import pipeline
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from typing import Final
from telegram import Update
from telegram.ext import Updater,Application,CommandHandler, MessageHandler,filters,ContextTypes
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from telethon import TelegramClient, events
from pydub import AudioSegment
from keras.models import load_model
from PIL import Image
from deepface import DeepFace
from fer import FER
from pydantic import BaseModel, Field

import replicate
import os
import sys
import random
import datetime
import time
import telegram
import requests
import telebot
import logging
import torch
import shutil
import cv2
import torch.nn as nn
import pandas as pd
import urllib.request
import speech_recognition as sr
import numpy as np
import audiosegment
import pydub
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
# api, tokens of the bots and the groq AI user:
groq_key=<groq_key>
BOT_USERNAME =<your_BOT_USERNAME>
TOKEN_TELEGRAM = <your_TOKEN_TELEGRAM>
TOKEN_IMAGE_BOT = "r8_E3392E8pR4yuG4ICKpYWNTiVWEm2bCn3SucFf"
URL = f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/getUpdates"

OFFSET = 0
row_index = 0
api = replicate.client.Client(api_token=TOKEN_IMAGE_BOT)
bot = telebot.TeleBot(TOKEN_TELEGRAM)
# prediction model data table:
df_prediction = pd.DataFrame(columns=['user id', 'sentiment', 'username', 'message'])
# select the emotion detection model in text:
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')
# select the emotion detection model in photo:
emotion_detector = FER(mtcnn=True)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
#class
class Responser_Groq(BaseModel):
    message: str = Field(description="the telegram message you need to classify and response accordenly")
    response: str = Field(description="the therapy response based on the telegram message")
#Functions

def send_goodbye_message(chat_id, TOKEN_TELEGRAM):
    response = "Goodbye!"
    base_url = f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
    requests.get(base_url)

def classify_emotion(input_text):
    """Classify the emotion of the given input text."""
    # Preprocess the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
    prediction = model.predict(padded_input_sequence)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])[0]
    return predicted_label
    #classify the emotion

def get_psychological_response(emotion):
    """Generate a psychological response based on the classified emotion."""
    responses = {
        'neutral': "Feeling neutral can be a peaceful place or a moment of rest. It’s okay to take this time just to be.",
        'joy': "It's wonderful to hear that you're feeling joyous! Cherish these moments—they're precious. Try to recognize what specifically is making you feel this joy, and consider how you might incorporate more of that into your daily routine.",
        'sadness': "It's completely normal to feel sad sometimes. Allow yourself to feel these emotions; you're not alone in this. Think about what might be causing your sadness and consider small actions that might brighten your day or provide comfort. you're not alone in this. Think about what might be causing your sadness and consider some of these actions to help you feel better: take a walk, listen to your favorite music, or talk to someone you trust about how you feel.",
        'fear': "It can be really challenging to face fear. Remember, it’s okay to feel scared. Learning to view and cope with the feared object or situation differently can be transformative. Try to challenge your worries, tolerate uncomfortable feelings, and build confidence in managing your thoughts.",
        'surprise': "Life is full of surprises, and it’s alright to feel taken aback. Take a moment to steady yourself.",
        'anger': "Feeling anger is a natural response. In the heat of the moment, try this breathing technique: Stop and focus on your breath, breathe in for a count of 7, breathe out for a count of 11, and continue for at least a minute. This can help activate your 'rest and digest' system, clearing your mind and helping you respond from a relaxed place.",
        'shame': "Dealing with shame can be deeply challenging, but addressing the root causes of these feelings can be enlightening. Focus on understanding the connection between your thoughts, emotions, and behaviors. Consider the automatic thoughts that pass unnoticed, like 'I don’t deserve love if I make a mistake.' Learning to be more self-compassionate and less self-critical is a crucial step in this journey.",
        'disgust': "Disgust can be intense. It’s important to acknowledge how you feel, as this can lead to deeper insights."
    }
    return responses.get(emotion, "I'm here to support you, even if your feelings are hard to define right now.")

    #return an appropriate psychological response
def respond_to_text(input_text):
    """Process the input text, classify its emotion, and return an appropriate psychological response."""
    emotion = classify_emotion(input_text)
    if emotion != 'neutral':
       response = get_psychological_response(emotion)
       base_url = f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
       requests.get(base_url)
    else:
        groq_key=<groq_key>

        chat = ChatGroq(temperature=1, groq_api_key=groq_key, model_name="mixtral-8x7b-32768")
        human = """please receive the message:{message} and response as LLM like chatgpt . use a JSON output with this schema:
                      "response":"string"
                       answer ONLY with JSON output with the schema above.
                   """
        system = """answer as a therapy  """

            # Set up a parser + inject instructions into the prompt template.
        parser = JsonOutputParser(pydantic_object=Responser_Groq)
        prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

        chain = prompt | chat | parser

        response = chain.invoke({"message":f"{message}"} )
        base_url = f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response['response']}"
        requests.get(base_url)
    return response
#``````


In [ ]:
condition = True
itr = 0

while condition:
    if itr<100:
        res = requests.get(URL, params={"offset": OFFSET})
        updates = res.json()["result"]
        itr += 1
        for update in updates:
            OFFSET = update["update_id"] + 1
            # Assume text messages for simplicity
            message_text = update.get("message", {}).get("text", "")
            message_voice = update.get("message", {}).get("voice", "")
            message_photo = update.get("message", {}).get("photo", "")
            chat_id = update.get("message", {}).get("chat", {}).get("id", 0)
            print(message_text)
            print(message_voice)
            print(message_photo)

              # if message_text =='message_voice'
            if (message_text == '' or message_text == None) and (message_photo == '' or message_photo == None):
              file_id = updates[0]['message']['voice']['file_id']
                                              # this code snippet retrieves the details of a voice message from
                                              # the Telegram API using the file ID.
                                              # It constructs the API endpoint URL, makes the request,
                                              # and parses the JSON response to extract the relevant information.
              url = f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/getFile?file_id={file_id}"
              detail = requests.get(url)
              data = detail.json()
              file_path = data['result']['file_path']
              url_copy = f"https://api.telegram.org/file/bot{TOKEN_TELEGRAM}/{file_path}"
              voice = requests.get(url_copy)
                                              # this code snippet downloads a voice message from Telegram,
                                              # saves it as an OGG file,
                                              # and then converts it to a WAV file using the AudioSegment library.
              with open('voice_note.ogg', 'wb') as f:
                  f.write(voice.content)
              print('voice downloaded successfully!')
              file_path = '/content'
              audio = AudioSegment.from_file(os.path.join(file_path,'voice_note.ogg'))
              audio.export(os.path.join(file_path,'voice_note.ogg'), format="wav")
              print('voice converted successfully!')
              # Initialize the recognizer:

              r = sr.Recognizer()
              with sr.AudioFile(os.path.join(file_path,'voice_note.ogg')) as source:
                  audio = r.record(source)
              try:
                  text = r.recognize_google(audio)
                  message = text
                  full_detail = bot.get_me()
                  print(full_detail)

                  message = message.lower()
                    # usage functions:
                  response = respond_to_text(message)
                  print( response)
                  #base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"


                  df_prediction.loc[row_index] = [chat_id, emotion_labels[0]['label'], user_fullname  , message]
                  row_index += 1
                  # Here you would add your logic to store the message in Google Sheets or Drive
                  #print(f"Message from {chat_id}: {text}")
                  if 'stop' in message:
                    send_goodbye_message(chat_id, TOKEN_TELEGRAM)
              except sr.UnknownValueError:
                  response = "Sorry, I couldn't understand the audio."
                  print(f"Response from {chat_id}: {response['response'][0:]}")

                              #message_text='text'
            elif (message_voice == '' or message_voice == None) and (message_photo == '' or message_photo == None):
                message = message_text
                full_detail = bot.get_me()
                #print(full_detail)

                message = message.lower()
                if ('draw' in message):           #draw recognition
                      print(f"Message from {chat_id}: {message}")
                      output = api.run(
                                  "bytedance/sdxl-lightning-4step:727e49a643e999d602a896c774a0658ffefea21465756a6ce24b7ea4165eba6a",
                                              input={
                                                  "width": 1024,
                                                  "height": 1024,
                                                  "prompt": message,
                                                  "scheduler": "K_EULER",
                                                  "num_outputs": 1,
                                                  "guidance_scale": 0,
                                                  "negative_prompt": "worst quality, low quality",
                                                  "num_inference_steps": 4
                                                    }
                                                                            )
                      base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendPhoto?chat_id={chat_id}&photo={output[0]}"
                      requests.get(base_url)
                else:
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendPhoto?chat_id={chat_id}&photo={output[0]}"
                    requests.get(base_url)
                    response = respond_to_text(message)
                    print( response)
                    emotion_labels = emotion(message)
                    df_prediction.loc[row_index] = [chat_id, emotion_labels[0]['label'], user_fullname , message]
                    row_index += 1

            else:
              file_id = update.get("message", {}).get("photo", "")[0]['file_id']
              # this code snippet retrieves the details of a photo message from
              # the Telegram API using the file ID.
              # It constructs the API endpoint URL, makes the request,
              # and parses the JSON response to extract the relevant information.
              url = f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/getFile?file_id={file_id}"
              detail = requests.get(url)
              data = detail.json()

              file_path = data['result']['file_path']
              url_copy = f"https://api.telegram.org/file/bot{TOKEN_TELEGRAM}/{file_path}"
              photo = requests.get(url_copy)
              filename = 'image.png'
              response = requests.get(url_copy)
              with open(filename, 'wb') as f:
                f.write(photo.content)

              print('photo downloaded successfully!')
              # Here you would add your logic to store the message in Google Sheets or Drive
              # We want to check if this photo is a face photo
              # Send response to user
              try:
                  download_image = cv2.imread(filename)
                  # image_analysis = DeepFace.analyze(img_path = filename)
                  # print(image_analysis)
                  dominant_emotion, emotion_score = emotion_detector.top_emotion(download_image)
                  print(dominant_emotion)
                  user_firstname = update.get("message", {}).get("from", {}).get("first_name", "")
                  user_lastname = update.get("message", {}).get("from", {}).get("last_name", "")
                  user_fullname = user_firstname + ' ' + user_lastname
                  df_prediction.loc[row_index] = [chat_id, dominant_emotion, user_fullname , None]
                  row_index += 1
                  if dominant_emotion == 'happy':
                    response = "You look happy! do you want to tell me why?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == 'sad':
                    response = "You look sad! do you want to tell me why?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == 'neutral':
                    response = "I'm not sure how are you feeling right now, do you want to tell me something?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == 'surprise':
                    response = "You look surprised! do you want to tell me why?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == 'anger':
                    response = "You look angry! do you want to tell me why?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == 'disgust':
                    response = "You look disgusted! do you want to tell me why?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == 'fear':
                    response = "You look scared! do you want to tell me why?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == 'contempt':
                    response = "You look contempt! do you want to tell me why?"
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)
                  elif dominant_emotion == None:
                    response = "I didn't get image of your face! please send an accurate photo of your face, so I can classify the emotion "
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={response}"
                    requests.get(base_url)

              except Exception as e:
                    print(f"Error downloading image: {e}")
                    base_url=f"https://api.telegram.org/bot{TOKEN_TELEGRAM}/sendMessage?chat_id={chat_id}&text={e}"
                    requests.get(base_url)



              # user_firstname = update.get("message", {}).get("from", {}).get("first_name", "")
              #  user_lastname = update.get("message", {}).get("from", {}).get("last_name", "")
              #  user_fullname = user_firstname + ' ' + user_lastname




[{'file_id': 'AgACAgQAAxkBAAIC52Yw4pKHoY4dilI3m3VACU6ApA6PAAINvzEbu-aJUcTufh_aIQjfAQADAgADcwADNAQ', 'file_unique_id': 'AQADDb8xG7vmiVF4', 'file_size': 1338, 'width': 67, 'height': 90}, {'file_id': 'AgACAgQAAxkBAAIC52Yw4pKHoY4dilI3m3VACU6ApA6PAAINvzEbu-aJUcTufh_aIQjfAQADAgADbQADNAQ', 'file_unique_id': 'AQADDb8xG7vmiVFy', 'file_size': 18799, 'width': 240, 'height': 320}, {'file_id': 'AgACAgQAAxkBAAIC52Yw4pKHoY4dilI3m3VACU6ApA6PAAINvzEbu-aJUcTufh_aIQjfAQADAgADeAADNAQ', 'file_unique_id': 'AQADDb8xG7vmiVF9', 'file_size': 80418, 'width': 600, 'height': 800}, {'file_id': 'AgACAgQAAxkBAAIC52Yw4pKHoY4dilI3m3VACU6ApA6PAAINvzEbu-aJUcTufh_aIQjfAQADAgADeQADNAQ', 'file_unique_id': 'AQADDb8xG7vmiVF-', 'file_size': 155098, 'width': 960, 'height': 1280}]
photo downloaded successfully!
None


[{'file_id': 'AgACAgQAAxkBAAIC6WYw4ql4aDBjQ333dj-6PqZdEN3bAAITvzEbu-aJUdLq8WE6myLCAQADAgADcwADNAQ', 'file_unique_id': 'AQADE78xG7vmiVF4', 'file_size': 1327, 'width': 67, 'height': 90}, {'file_id': 'AgACAgQAAxkBAA

KeyboardInterrupt: 

-------------------------------